# Map Styles

This notebook shows how to use different map styles.

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

import geopandas
import contextily

In [ ]:
%matplotlib inline

import matplotlib as mpl
mpl.rcParams["figure.facecolor"]  = '0.95'
mpl.rcParams["figure.figsize"]  = (10,10)

## Prepare some data

Let's get some test data from the Danish Addresses Web API, DAWA.
We will pick the locations on Vestmannagade, a short street in Copenhagen.

In [ ]:
r = requests.get('https://dawa.aws.dk/adresser?postnr=2300&vejnavn=Vestmannagade&struktur=mini')
address_list = r.json()

In [ ]:
df = pd.DataFrame.from_records(address_list, columns=['x','y'])
df.head()

In [ ]:
# DAWA data have x, y WGS 84 format, using the EPSG:4326 coordinate references system (crs)

gdf = geopandas.GeoDataFrame(df,
                             crs='EPSG:4326',
                             geometry=geopandas.points_from_xy(df.x, df.y)
                             )

print(gdf.head())


## Hello, world
Let's plot these using the built-in low-res map.

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
ax = world[world.name == 'Denmark'].plot(color='white', edgecolor='black')
gdf.plot(ax=ax, color='red')
plt.show()

## Adding Base Maps
This will look much nicer. `contextily` provides convenient access to a range of map tiles.

In [ ]:
def plot_with_style(geopandas_dataframe, source):
    ax = geopandas_dataframe.plot(color='red', alpha=0.5)
    contextily.add_basemap(ax, crs=gdf.crs.to_string(),
                           source=source)
    ax.set_title(source.name)

In [ ]:
plot_with_style(gdf, contextily.providers.OpenStreetMap.Mapnik)

In [ ]:
plot_with_style(gdf, contextily.providers.Stamen.Toner)

In [ ]:
plot_with_style(gdf, contextily.providers.Wikimedia)

In [ ]:
plot_with_style(gdf, contextily.providers.CartoDB.Voyager)

## Zooming out a bit
Let's zoom out to see a bit of the neighbourhood.
We can use the `buffer` function to get the geometry of an area around the addresses.

In [ ]:
buffer = gdf.buffer(distance=0.002)
plot_with_style(buffer, contextily.providers.Wikimedia)

In [ ]:
ax = gdf.plot(color='red', alpha=0.5)

# We can use normal matplotlib manipulation of the x and y limits
# to zoom out to the bounding box of the buffer
ax.set_xlim(buffer.total_bounds[0], buffer.total_bounds[2])
ax.set_ylim(buffer.total_bounds[1], buffer.total_bounds[3])

contextily.add_basemap(ax, crs=gdf.crs.to_string(),
                       source=contextily.providers.Wikimedia)

### Arranging maps in grids
The normal matplotlib subplot features are available since we can plot to an axis.

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True,
                        subplot_kw={
                            'xlim': (buffer.total_bounds[0], buffer.total_bounds[2]),
                            'ylim': (buffer.total_bounds[1], buffer.total_bounds[3])
                        },
                        figsize=(10,10))

for (ax, tile_source) in zip(axs.flat,
                       [contextily.providers.Stamen.Watercolor,
                        contextily.providers.Stamen.Toner,
                        contextily.providers.Wikimedia,
                        contextily.providers.CartoDB.Voyager]):
    ax.set_title(tile_source.name)
    gdf.plot(ax=ax, color='r', alpha=0.5)
    contextily.add_basemap(ax, crs=gdf.crs.to_string(), source=tile_source)

fig.suptitle('Subplots with different basemaps')